### Question 3: Interview Scheduling

In [11]:
# import Pkg; Pkg.add("NamedArrays")

In [10]:
using JuMP, NamedArrays, GLPK


availability = [
0     0     1     1     0     0     0     1     1     0     0     0     0
0     1     1     0     0     0     0     0     1     1     0     0     0
0     0     0     1     1     0     1     1     0     1     1     1     1
0     0     0     1     1     1     1     1     1     1     1     1     0
0     0     0     0     0     0     1     1     1     0     0     0     0
0     1     1     0     0     0     0     0     1     1     0     0     0
0     0     0     1     1     1     1     0     0     0     0     0     0
1     1     0     0     0     0     0     0     0     0     1     1     1
1     1     1     0     0     0     0     0     0     1     1     0     0
0     0     0     0     0     0     0     1     1     0     0     0     0
0     0     0     0     0     0     1     1     1     0     0     0     0
1     1     0     0     0     1     1     1     1     0     0     1     1
1     1     1     0     1     1     0     0     0     0     0     1     1
0     1     1     1     0     0     0     0     0     0     0     0     0
1     1     0     0     1     1     0     0     0     0     0     0     0
]

TIME = collect(1:13)
NAMES = [:Manuel,:Luca,:Jule,:Michael,:Malte,:Chris,:Spyros,:Mirjam,:Matt,:Florian,:Josep,:Joel,:Tom,:Daniel,:Anne ]
TIMESTR = ["10:00","10:20","10:40","11:00","11:20","11:40","lunch","1:00","1:20","1:40","2:00","2:20","2:40"]

# Creating a NamedArray of the availability data
times = NamedArray( availability, (NAMES,TIME), ("NAME","TIME"))

# With a NamedArray, it is possible to use symbols as indices (although you don't have to use this)
# For example, it is possible to write

println(times[:Luca,1])

# or

println(sum(times[i,1] for i in NAMES))

0
5


In [11]:
#Model:
model = Model(GLPK.Optimizer)
@variable(model, x[NAMES, TIME], Bin)

#Constraint 1: Every time slot, except lunch must have exactly one assigned employee
for t in TIME
    if t != 7 # Exclude lunch slot
        @constraint(model, sum(x[i, t] for i in NAMES) == 1)
    end
end

#Constraint 2: At 10AM, at least one of Mirjam or Matt must be present
@constraint(model, x[:Mirjam, 1] + x[:Matt, 1] >= 1)

#Constraint 3: Each employee is assigned to at most one slot
for i in NAMES
    @constraint(model, sum(x[i, t] for t in TIME) <= 1)
end

#Constraint 4: Employees can only be assigned to slots when available
for i in NAMES, t in TIME
    @constraint(model, x[i, t] <= times[i, t])
end

optimize!(model)

In [12]:
#Feasibility
if termination_status(model) == MOI.OPTIMAL
    println("A feasible schedule exists.")
else
    println("No feasible schedule exists.")
end

A feasible schedule exists.


In [13]:
# a helper for printing schedule

assignment = [ Int(JuMP.value(x[i,j])) for i in NAMES, j in TIME ]
                                                
println("INTERVIEW SCHEDULE:")
for (i,t) in enumerate(TIMESTR)
    print(t, " -- meet with ")
    for j = findall(assignment[:,i] .!= 0)
        print(NAMES[j]," ")
    end
    println()
end

INTERVIEW SCHEDULE:
10:00 -- meet with Mirjam 
10:20 -- meet with Luca 
10:40 -- meet with Manuel 
11:00 -- meet with Jule 
11:20 -- meet with Anne 
11:40 -- meet with Spyros 
lunch -- meet with 
1:00 -- meet with Malte 
1:20 -- meet with Chris 
1:40 -- meet with Matt 
2:00 -- meet with Michael 
2:20 -- meet with Joel 
2:40 -- meet with Tom 
